<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FPreprocessing/Preprocessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install**

# **Import**

In [1]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time
import math
import random

from sklearn.model_selection import train_test_split

## Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation
!ls -al

/content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation
total 12550
drwx------ 2 root root     4096 Sep 14 08:50 'Case Report'
drwx------ 2 root root     4096 Aug 16 08:53  Dataset
drwx------ 2 root root     4096 Sep 25 16:33 '# Lagacy'
drwx------ 2 root root     4096 Aug 17 05:49  Log
-rw------- 1 root root  2078929 Oct  2 07:34 '[Model Tester] Down Sampling.ipynb'
-rw------- 1 root root    87987 Oct  4 16:13  Preprocessor.ipynb
drwx------ 2 root root     4096 Sep 14 06:04 'Raw Dataset'
-rw------- 1 root root 10662546 Oct  4 12:42 'Surgical-Wound UNet.ipynb'


# **Grobal variable**

In [4]:
# Path (Absolute path in essential)
MODEL_PATH = "/content/gdrive/MyDrive/Models/Surgical-Wound_Segmentation"

RAW_DATA_PATH = "/Raw Dataset"

WOUND_TRAIN_PATH = "/Wound/train"
WOUND_TEST_PATH = "/Wound/test"
CVC_INPUT_PATH = "/CVC-clinicDB/Original"
CVC_GT_PATH = "/CVC-clinicDB/Ground Truth"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 224
NORM_INPUT_H_SIZE = 224

# **Function**

In [5]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [6]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

<br>
<br>
<br>
<br>
<br>
<br>

# ***Preprocess Performace Compare Test***


---

<br>

***공통사항***
* 원본 이미지 크기 = 224 * 224
* epoch = 100
* batch = 8
> UNet batch size = 4<br>
> Wound dataset batch size = 8
* lr = 1e-3
* Train:Val = 7:3
* Used model = Polar Res UNet++ (ISIC 2018 dice scroe rank no.1)

<br>

***주의사항***
* 이미지의 사이즈는 정규화 되어야 한다.
* Convolution에서 특징점을 잘 추출하기 때문에 색상영역이나, 스무딩/샤프닝 같은 필터 처리는 되려 성능저하 요소가 될 수 있다. 

<br>

***참고사항***
* CNN 대표 모델들의 네트워크 입력 사이즈는 224 * 224
* 이미지의 사이즈가 큰 경우에, Overlap-Tite(down sampling) 전략으로 한 이미지를 나눠서 학습시킨다.

<br>

***현재 결과***

---




<br>
<br>
<br>
<br>
<br>
<br>

## [***Control Group***](https://www.nature.com/articles/s41598-020-78799-w)
> [Train code](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/train.py)
> <br>
> [Data generator](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/utils/io/data.py#L10)
> <br>
> <br>
> Raw dataset에 대한 전처리 없이, load 후 model에 들어가기 전에 Normalize
> <br>
> * 현재까지 Case 2가 제일 성능 좋음

```
model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
```

### Performance

#### **Color**

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 500 / 500

TRAIN || EPOCH 0500 | BATCH  073 /  073 | DICE LOSS  0.0389
VALID || EPOCH 0500 | BATCH  032 /  032 | DICE LOSS  0.1028

Best loss in Train (epoch 494, loss 0.0386)
Best loss in Val (epoch 487, loss 0.0985)

Model train / validate time =  10341초 / 172분 / 2시간 52분
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1vFqsvObms3O61x-QvoE_7TFXrqVi_Hl_"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1emnRmhENDMZHoMM8nYuTT9R1dmZ2Uc1E"  width = 1080>

### Arrange dataset
> Not change anything about input image(scar)

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    wound = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    wound = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    wound = cv2.imread(gt)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

### ※ Effect of dice score on resize

#### Control group size * 2

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 500 / 500

TRAIN || EPOCH 0500 | DICE LOSS  0.0437
VALID || EPOCH 0500 | DICE LOSS  0.1240

Best loss in Train (epoch 496, loss 0.0398)
Best loss in Val (epoch 231, loss 0.1158)

Model train / validate time =  27510초 / 459분 / 7시간 39분
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=18siVBDjewA7RSI9Z1gfanxA3-zBID774"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1-acCBy0AGgXgZBvEbRqoRCe3ZRDeGR1M"  width = 1080>

#### Control group size * 1/2

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 500 / 500

TRAIN || EPOCH 0500 | DICE LOSS  0.0501
VALID || EPOCH 0500 | DICE LOSS  0.1138

Best loss in Train (epoch 500, loss 0.0501)
Best loss in Val (epoch 296, loss 0.1071)

Model train / validate time =  4381초 / 73분 / 1시간 13분
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1IacXpF9HLNjcEQXNob_GmRsxcOckF2CZ"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1Q6jdU-mm9kKZ0n-5HWl2oMhzw9yHtxdp"  width = 1080>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    wound = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Resize (2배)
    wound = cv2.resize(wound,
                        (0, 0),
                        fx=2.0,
                        fy=2.0,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=2.0,
                             fy=2.0,
                             interpolation=cv2.INTER_CUBIC)
    
    """
    # Resize (1/2배)
    image = cv2.resize(image,
                        (0, 0),
                        fx=0.5,
                        fy=0.5,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                          (0, 0),
                          fx=0.5,
                          fy=0.5,
                          interpolation=cv2.INTER_CUBIC)
    """

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), wound):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    wound = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # Resize (2배)
    wound = cv2.resize(wound,
                        (0, 0),
                        fx=2.0,
                        fy=2.0,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=2.0,
                             fy=2.0,
                             interpolation=cv2.INTER_CUBIC)
    
    """
    # Resize (1/2배)
    image = cv2.resize(image,
                        (0, 0),
                        fx=0.5,
                        fy=0.5,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                          (0, 0),
                          fx=0.5,
                          fy=0.5,
                          interpolation=cv2.INTER_CUBIC)
    """

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), wound):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    wound = cv2.imread(gt)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # Resize (2배)
    wound = cv2.resize(wound,
                        (0, 0),
                        fx=2.0,
                        fy=2.0,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=2.0,
                             fy=2.0,
                             interpolation=cv2.INTER_CUBIC)
    
    """
    # Resize (1/2배)
    image = cv2.resize(image,
                        (0, 0),
                        fx=0.5,
                        fy=0.5,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                          (0, 0),
                          fx=0.5,
                          fy=0.5,
                          interpolation=cv2.INTER_CUBIC)
    """

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), wound):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

### ※ Down sampling

* Case 4 이미지에서 패딩 제거 후 비율을 지킨후 더 작은 사이드를 224 까지 resize
* 정규화 관련 코드 변경 ( /255.0 >> cv2.normalize )
* ground-truth 이미지 resizing 후 threshold로 흐릿한 경계선 선명하게 처리
* resize 방법 CUBIC -> LANCZOS4

#### Color

```
Input size =  torch.Size([8, 3, 224, 224])

Epoch =  / 500

TRAIN || EPOCH 0500 | DICE LOSS  
VALID || EPOCH 0500 | DICE LOSS  

Best loss in Train ()
Best loss in Val ()

Model train / validate time = 
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1IacXpF9HLNjcEQXNob_GmRsxcOckF2CZ"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1Q6jdU-mm9kKZ0n-5HWl2oMhzw9yHtxdp"  width = 1080>

<br>
<br>
<br>
<br>
<br>
<br>

## ***Experimental Group 1***
* Color space modify

### Case 1

1. Delete Black Boundary(Padding)
2. Equlize histogram of l space in Lab color space
3. CLAHE histogram of b space in Lab color space
4. Restore padding

#### **Color**




```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 500 / 500

TRAIN || EPOCH 0500 | DICE LOSS  0.0611
VALID || EPOCH 0500 | DICE LOSS  0.1206

Model train / validate time =  11000초 / 180분 / 3시간
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1xPdp48StgmBU8WOZUYbJJYfrOF1i0bN-"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1lAjHCwRc7IxHPxJ7K9xBTPfzam4ZNpkt"  width = 1080>

#### Preprocess

In [7]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)
    
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    
    # Equalize histogram used by LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)
    
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    
    # Equalize histogram used by LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(gt)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)
    
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    
    # Equalize histogram used by LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>
<br>

## ***Experimental Group 2***

* Resize
* Move

### Case 1

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (224, 224)

#### **Color**




```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 500 / 500

TRAIN || EPOCH 0500 | BATCH  073 /  073 | DICE LOSS  0.0279
VALID || EPOCH 0500 | BATCH  032 /  032 | DICE LOSS  0.1612

Best loss in Train =  0.0269
Best loss in Validation =  0.1278

Model train / validate time =  15063초 / 251분 / 4시간 18분
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=14Y94sANIuQ_qiU75ewWst-gJNp9REwIs"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1dItwQrHA16VV7GbkIl_NuHG5ouOkrpV-"  width = 1080>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in train_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    gt_image = cv2.resize(gt_image, 
                          (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    gt_image = cv2.resize(gt_image, 
                          (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed scar image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224)
    # Resize input image
    image = cv2.resize(image,
                       (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                       interpolation=cv2.INTER_CUBIC)
    
    # Resize label image
    gt_image = cv2.resize(gt_image,
                          (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                          interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 2

1. Delete Black Boundary(Padding)
2. Pad the rest part to (224, 224) based on the middle of the image

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

Epoch =  500 / 500

TRAIN || EPOCH 0500 | DICE LOSS  0.0450
VALID || EPOCH 0500 | DICE LOSS  0.1118

Model train / validate time =  약 4시간
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1n4igpVQjAgrQvNjeVVKMQwaoC5jUgyU9"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1DzoG3HOw6C_7K-v_GT5oGd_bNcWESC9s"  width = 1080>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])


    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")

    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Pad image to (224, 224)
    # Calculate padding size
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    # Modify error
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
    
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Resizing
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 3

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) in the direction of right/down 

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 500 / 500

TRAIN || EPOCH 0500 | BATCH  073 /  073 | DICE LOSS  0.0847
VALID || EPOCH 0500 | BATCH  032 /  032 | DICE LOSS  0.1836

Best loss in Train =  0.0324
Best loss in Validation =  0.1328

Model train / validate time =  12223초 / 203분 / 3시간 28분
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1ThzFQptjGhNH-ZVk8LWgoaqs7Kr5O0lO"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=10EughBXJqkl0FBUBeDqefXLNk0ttGDIz"  width = 1080>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access wound image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Calculate padding size
    pad_bottom = NORM_INPUT_H_SIZE - height
    pad_right = NORM_INPUT_W_SIZE - width
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               0,
                               pad_bottom,
                               0,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 4

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (224, 224) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) based on the middle of the image

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

Epoch = 500 / 500

TRAIN || EPOCH 0500 | BATCH  073 /  073 | DICE LOSS  0.0291
VALID || EPOCH 0500 | BATCH  032 /  032 | DICE LOSS  0.1664

Best loss in Train =  0.0279
Best loss in Validation =  0.1341

Model train / validate time =  12279초 / 204분 / 3시 24분
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1vsqPr4o5Fn1Km0a3R6yxuiCfEfCRrsP9"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1I-bGbu6bqvivhCmEbp9ukF38RepkNSZS"  width = 1080>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    if resize_criteria == 0: # criteria is width
        pad_height = round((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    # Calculate padding size
    elif resize_criteria == 1: # criteria is heigth
        pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Print resize error
    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    if resize_criteria == 0: # criteria is width
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    # Calculate padding size
    elif resize_criteria == 1: # criteria is heigth
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Print resize error
    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access wound image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (224, 224) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = NORM_INPUT_W_SIZE / width
    resize_scale_h = NORM_INPUT_H_SIZE / height

    # Decide resize criteria between width and height
    resize_criteria = -1
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
        resize_criteria = 0
    else:
        resize_scale = resize_scale_h
        resize_criteria = 1

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
 
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    if resize_criteria == 0: # criteria is width
        pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

        pad_top = pad_height
        pad_bottom = pad_height
        
        total_height = height + (pad_height * 2)

        if total_height > NORM_INPUT_H_SIZE:
            pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
        elif total_height < NORM_INPUT_H_SIZE:
            pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    # Calculate padding size
    elif resize_criteria == 1: # criteria is heigth
        pad_width = int((NORM_INPUT_W_SIZE - width) / 2)

        pad_left = pad_width
        pad_right = pad_width

        total_width = width + (pad_width * 2)

        if total_width > NORM_INPUT_W_SIZE:
            pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
        elif total_width < NORM_INPUT_W_SIZE:
            pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    # Print resize error
    else:
        print("Resize Error, [ocurred index] = ", idx)
        break
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1

<br>
<br>
<br>
<br>
<br>

### Case 5

1. Delete Black Boundary(Padding)
2. Resize Wound Image to (204, 204) in proportion to the maximum ratio
3. Pad the rest part to (224, 224) based on the middle of the image

#### **Color**


```
Input size =  torch.Size([8, 3, 224, 224])

Epoch =  / 500

TRAIN || EPOCH 0484 | BATCH  073 /  073 | DICE LOSS  0.0325
VALID || EPOCH 0484 | BATCH  032 /  032 | DICE LOSS  0.1715

Best loss in Train =  0.0287
Best loss in Validation =  0.1366

Model train / validate time =  12514초 / 209분 / 3시간 29분
```

##### ***Train/Val***

<br>

<img src = "https://drive.google.com/uc?id=1vgFf4CzvGxqskkQPk59-tq1YQCPUAHq1"  width = 1080>

<br>

---

<br>

<img src = "https://drive.google.com/uc?id=1O4HrU09VpYzWX5UztR7xnDRBcOxiZ_42"  width = 1080>

#### Preprocess

In [ ]:
# Set load image dir path
train_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + IMAGES_PATH
train_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TRAIN_PATH + LABELS_PATH
test_image_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + IMAGES_PATH
test_gt_path = MODEL_PATH + RAW_DATA_PATH + WOUND_TEST_PATH + LABELS_PATH

# Load raw input images for Train
inputs_train = os.listdir(train_image_path)
inputs_train.sort()

# Load raw label images for Train
gts_train = os.listdir(train_gt_path)
gts_train.sort()

# Load raw input images for Test
input_test = os.listdir(test_image_path)
input_test.sort()

# Load raw label images for Test
gt_test = os.listdir(test_gt_path)
gt_test.sort()

# Split train:val:test = 7:3
input_train, input_val, gt_train, gt_val = train_test_split(inputs_train, gts_train, test_size=0.3, random_state=1)

# Merge train/val/test set
train_dataset = dict(zip(input_train, gt_train))
val_dataset = dict(zip(input_val, gt_val))
test_dataset = dict(zip(input_test, gt_test))

# ================================ Train ================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in train_dataset.items(): 
    # Access wound image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access ground-truth image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (204, 204) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = (NORM_INPUT_W_SIZE - 20) / width
    resize_scale_h = (NORM_INPUT_H_SIZE - 20) / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)

    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    pad_height = round((NORM_INPUT_H_SIZE - height) / 2)
    pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width
        
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)

    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(train_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write train wound image")
    if not cv2.imwrite(os.path.join(train_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write train ground-truth image")
        
    idx += 1

# ============================== Validation ==============================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in val_dataset.items(): 
    # Access input image
    os.chdir(train_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(train_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (204, 204) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = (NORM_INPUT_W_SIZE - 20) / width
    resize_scale_h = (NORM_INPUT_H_SIZE - 20) / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)
    
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    pad_height = round((NORM_INPUT_H_SIZE - height) / 2)
    pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width
        
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)

    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(val_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write val wound image")
    if not cv2.imwrite(os.path.join(val_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write val ground-truth image")

    idx += 1

# ================================= Test =================================
# Init image index
idx = 0

# Create preprocessed wound image
for input, gt in test_dataset.items(): 
    # Access input image
    os.chdir(test_image_path)
    image = cv2.imread(input)

    # Access label image
    os.chdir(test_gt_path)
    gt_image = cv2.imread(gt, cv2.IMREAD_GRAYSCALE)

    # 1. Cut black boundary on value image
    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find contours
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    # Crop ROI
    image = image[y:y+height, x:x+width]
    gt_image = gt_image[y:y+height, x:x+width]

    # 2. Resize image to (204, 204) in proportion to the maximum ratio
    # Calculate resize ratio
    resize_scale_w = (NORM_INPUT_W_SIZE - 20) / width
    resize_scale_h = (NORM_INPUT_H_SIZE - 20) / height

    # Decide resize criteria between width and height
    if resize_scale_w < resize_scale_h:
        resize_scale = resize_scale_w
    else:
        resize_scale = resize_scale_h

    # Resize
    image = cv2.resize(image,
                        (0, 0),
                        fx=resize_scale,
                        fy=resize_scale,
                        interpolation=cv2.INTER_CUBIC)
    gt_image = cv2.resize(gt_image,
                          (0, 0),
                          fx=resize_scale,
                          fy=resize_scale,
                          interpolation=cv2.INTER_CUBIC)
    
    # Get height and width
    height, width = gt_image.shape

    # 3. Pad image to (224, 224)
    # Init padding val
    pad_top = 0
    pad_bottom = 0
    pad_left = 0
    pad_right = 0

    # Calculate padding size
    pad_height = round((NORM_INPUT_H_SIZE - height) / 2)
    pad_width = round((NORM_INPUT_W_SIZE - width) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width
        
    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)

    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)
    
    # Padding
    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    gt_image = cv2.copyMakeBorder(gt_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    if not cv2.imwrite(os.path.join(test_dir, f'wound_{idx:03d}.png'), image):
        raise Exception("Could not write test wound image")
    if not cv2.imwrite(os.path.join(test_dir, f'gt_{idx:03d}.png'), gt_image):
        raise Exception("Could not write test ground-truth image")
        
    idx += 1